# **Calibration of a complete Mueller polarimeter**
*Author*: @ mariajloperaa

## This notebook contains all the elements to calibrate and use the automatic Mueller polarimeter. The calibration assumes a vertical polarized light source.
The polarimeter consists on a polarized light source, on a PSG a $\lambda/2$ followed by a $\lambda/4$, then after the sample a $\lambda/4$ followed by a linear polarizer.

The **initial calibration** consists on finding the steps per revolution or the steps per degree of each motor.


In [117]:
from polarimeter_functions import *
import time
import serial
import tisgrabber as IC
import numpy as np
import pandas as pd
import cv2 as cv

## *Default parameters*

In [316]:
step = 50
iterations = 3000

# Initialize serial port
ser = serial.Serial('COM8', 115200)
# Initialize camera
Camera = IC.TIS_CAM()


# Calibration 1: *Calibration of the steps required to move 1 degree.*

## Calibration 1, MOTOR 1

In [318]:
# motor_1 = calibration_spr(Camera,1,step,iterations)
motor_1 = pd.read_csv('Calibration_motor_1.csv',index_col=0)
d_m1 = steps_per_degree(motor_1,1)

The motor takes 17750.0 steps to run 360 degrees. Taking 49.30555555555556 steps to move 1 degree


49.30555555555556

## Calibration 1, MOTOR 2

In [ ]:
# motor_2 = calibration_spr(Camera,1,step,iterations)
motor_2 = pd.read_csv('Calibration_motor_2.csv',index_col=0)
d_m2 = steps_per_degree(motor_2,2)

## Calibration 1, MOTOR 3

In [ ]:
# motor_3 = calibration_spr(Camera,1,step,iterations)
motor_3 = pd.read_csv('Calibration_motor_3.csv',index_col=0)
d_m3 = steps_per_degree(motor_3,3)

## Calibration 1, MOTOR 4

In [ ]:
# motor_4 = calibration_spr(Camera,1,step,iterations)
motor_4 = pd.read_csv('Calibration_motor_4.csv',index_col=0)
d_m4 = steps_per_degree(motor_4,4)

# Calibration 2: *Finding the zero or fast axis of each of the polarization elements*

 Linear polarizer PSA located at **90 degrees**

In [ ]:
calibrate_element(Camera,motor_4,d_m4)


*PSG* $\lambda /2$'s fast axis aligned to the polarization axis of the source, *zero degrees*


In [ ]:
calibrate_element(Camera,motor_1,d_m1)

*PSA* $\lambda /4$'s fast axis aligned to the polarization axis of the source, *zero degrees*


In [ ]:
calibrate_element(Camera,motor_2,d_m2)

*PSA* $\lambda /4$'s fast axis aligned to the polarization axis of the source, *zero degrees*

In [ ]:
calibrate_element(Camera,motor_3,d_m3)

The intensity should remain at zero, then the linear polarized is shifted again to its zero.

In [ ]:
measure_irradiance(Camera)
move_motor(motor_4, d_m4 * 90)
measure_irradiance(Camera)

# Mueller matrix measurement

## Initial development just for bright field


In [ ]:
save_path = ''
angles = np.array([[0, 0, 90, 90], [0, 0, 90, 0], [0, 0, -45, 45], [0, 0, 45, -45],[0, 0, 0, -45], [0, 0, 0, 45],
                   [45, 90, 90, 90], [45, 90, 90, 0], [45, 90, -45, 45],[45, 90, 45, -45], [45, 90, 0, -45], [45, 90, 0, 45],
                   [-22.5, -45, 90, 90],[-22.5, -45, 90, 0], [-22.5, -45, -45, 45], [-22.5, -45, 45, -45], [-22.5, -45, 0, -45],[-22.5, -45, 0, 45],
                   [22.5, 45, 90, 90], [22.5, 45, 90, 0], [22.5, 45, -45, 45], [22.5, 45, 45, -45], [22.5, 45, 0, -45], [22.5, 45, 0, 45],
                   [22.5, 90, 90, 90],[22.5, 90, 90, 0], [22.5, 90, -45, 45], [22.5, 90, 45, -45], [22.5, 90, 0, -45], [22.5, 90, 0, 45],
                   [45, 90, 90, 90], [45, 90, 90, 0], [45, 90, -45, 45], [45, 90, 45, -45], [45, 90, 0, -45], [45, 90, 0, 45], [0, 0, 0, 0]])


df = pd.DataFrame(columns=['Name','Counts'])
data = {}
names = [['HV', 'HH','HP','HM','HR','HL',
          'VV','VH','VP','VM','VR','VL',
          'MV','MH','MP','MM','MR','ML',
          'PV','PH','PP','PM','PR','PL',
          'LV','LH','LP','LM','LR','LL',
          'VV','VH','VP','VM','VR','VL','ref']]

for angle, name in zip(angles,names):
    move_motor(motor_4, d_m4 * angle[0])
    move_motor(motor_4, d_m4 * angle[1])
    move_motor(motor_4, d_m4 * angle[2])
    move_motor(motor_4, d_m4 * angle[3])

    time.sleep(1)
    image  = Camera.GetImage()
    data[name] = image # directly just for bright field
    cv.imwrite(image,f'{save_path}/{name}.png')
    df = df.append({'Name':name,'Counts':measure_irradiance(Camera)},ignore_index = True)

m00, m01, m02, m03, m10, m12, m11, m13, m21, m20, m23, m30, m31, m32, m33, m22 = mueller_mat(data)